In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation

This file creates the aggregate trade data.


In [2]:
trade_type = "imports"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
crl = ["darkblue","slategray","crimson"]

background = "#ffffff"

---
#### Grabe the trade data using the Census's API

In [4]:
def census_trade(url, trade_type, country, product_level):
    
    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m")
    # This is so I can call this correctly...
    
    if trade_type == "imports":
        
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        df[product_level] = df["NAICS"].astype(str)
        
        df.drop(["CON_VAL_MO", "NAICS", "COMM_LVL"], axis = 1, inplace = True)
        
    if trade_type == "exports":
    
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)

        df[product_level] = df["NAICS"].astype(str)
        
        df.drop(["ALL_VAL_MO", "NAICS", "COMM_LVL"], axis = 1, inplace = True)
        
    if trade_type == "country_exports":
    
        trade_label = "total_exports"
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)
        
        df = df[df.SUMMARY_LVL == "DET"]
        
        df.drop(["ALL_VAL_MO", "CTY_CODE", "SUMMARY_LVL"], axis = 1, inplace = True)
        
        df = df[df.CTY_NAME != "TOTAL FOR ALL COUNTRIES"]
        
    if trade_type == "country_imports":
    
        trade_label = "total_imports"
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        df = df[df.SUMMARY_LVL == "DET"]
        
        df.drop(["CON_VAL_MO", "CTY_CODE", "SUMMARY_LVL"], axis = 1, inplace = True)
        
        
    
    return df

---
### And this is the Product Plot

In [ ]:
end_use = "naics?get=NAICS,CON_VAL_MO,NAICS_LDESC"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=NA3"

url = url

df = census_trade(url, trade_type, "total", "naics")

df.time.max()

In [ ]:
df.head()

In [ ]:
#end_use = "hs?get=E_COMMODITY,ALL_VAL_MO,E_COMMODITY_SDESC"

#url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
#url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=HS2"

#url = url

#df = census_trade(url, trade_type, "total", "hs2")

#df.time.max()

In [ ]:
def growth_trade(foo):
    # what this function does is take a dataframe and create a relative 
    
    foo["growth"] = 100*((foo.total_imports/foo.total_imports.shift(12)) - 1)
    
    return foo

In [ ]:
def make_trade_time(df):
        
    foo = df.drop(labels = ["naics"], axis = 1).reset_index().pivot(index = "time", columns = "NAICS_LDESC")
    
    foo.drop(labels = ["index"], axis = 1, inplace = True)
    
    return foo

In [ ]:
grp = df.groupby(["naics"])

top_products = grp.agg({"total_imports":"sum"})

plist = list(top_products.sort_values(by = "total_imports", ascending = False).index.values)[0:9]

df = df[df.naics.isin(plist)]

In [ ]:
grp = df.groupby(["naics"])

productdf = grp.apply(growth_trade)

In [ ]:
def make_time_by_product(df, level):

    height = 533
    width = 625

    foobar = make_trade_time(df)
    
    if level == "total_imports":
        title = "By Product (NAICS 3), $ US Imports "
    
    if level == "growth":
        title = "By Product (NAICS 3), % Year over Year Change in US Imports"
    
    p = figure(plot_height=height, plot_width = width, x_axis_type="datetime",toolbar_location = 'below',
           tools = "box_zoom, reset, pan, xwheel_zoom", title = title, x_range = (dt.datetime(2019,1,1),dt.datetime(2020,6,1)) ) 

    numlines=len(foobar[level].columns)

    multi_line_source = ColumnDataSource({
        'xs': [foobar.index.values]*numlines,
        'ys': [foobar[level, name].values for name in foobar[level]],
        'label': [name for name in foobar["total_imports"].columns]})

    p.multi_line(xs= "xs",
                ys= "ys",
                line_width=2.5, line_alpha=0.45, line_color = "slategray",
                 hover_line_alpha=0.75, hover_line_width = 5,
                hover_line_color= "crimson", source = multi_line_source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)
            
    TIMETOOLTIPS = """
            <div style="background-color:#F5F5F5; opacity: 0.95; border: 5px 5px 5px 5px;">
            <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold"> @label
             </span>
             </div>
             <div style = "text-align:left;">"""
    
    if level == "total_imports":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{$0.0a}</span>   
            </div>
            </div>
            """
    if level == "growth":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{0.0}%</span>   
            </div>
            </div>
            """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS,  line_policy='nearest', formatters={'$data_x': 'datetime'}))
    p.title.text_font_size = '13pt'
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75
    p.border_fill_color = background 
    
    #tradewar_box = BoxAnnotation(left=dt.datetime(2018,7,1), right=dt.datetime(2019,10,11), fill_color='red', fill_alpha=0.1)
    #p.add_layout(tradewar_box)
    
    pandemic_box = BoxAnnotation(left=dt.datetime(2020,3,1), right=dt.datetime(2020,12,31), fill_color='red', fill_alpha=0.1)
    p.add_layout(pandemic_box)
    
    #p.yaxis.axis_label = 
    p.yaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = "13px"

    p.yaxis.minor_tick_line_color = None
    #p.y_range.start = 0
    
    #p.x_range.max_interval = dt.datetime(2020,6,1)
    
    #description = """Each line is a major product catagory in Annex 6-1 of The Agreement."""
    #p.add_layout(Title(text=description, text_font_style="normal", text_font_size="9pt"), 'below')
    #description = "Red marks the period of Section 301 tariffs and retaliation. Blue is period of agreement."""
    #p.add_layout(Title(text=description, text_font_style="normal", text_font_size="9pt"), 'below')
    
    div0 = Div(text = """Each line is a 3 digit NAICS catagory. Only the top 10 catagories are shown. Red is the start of the NBER
    determined recession and Covid-19 pandemic in the US.""", width=555, background = background )
    
    if level == "total_imports":
        p.yaxis.formatter = NumeralTickFormatter(format="($0. a)")
    
    p.outline_line_color = None
    
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    #p.WheelZoomTool.maintain_focus = False
    #print(p.y_range.end)
    #p = column(p,div0)

    return p

---
### This is the Country plot...

In [ ]:
end_use = "naics?get=CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01"

url = url

df = census_trade(url,"country_imports","foo", "foo")

df.time.max()

In [ ]:
grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum"})

clist = list(top_products.sort_values(by = "total_imports", ascending = False).index.values)[0:11]

df = df[df.CTY_NAME.isin(clist)]

In [ ]:
grp = df.groupby(["CTY_NAME"])

country_df = grp.apply(growth_trade)

In [ ]:
country_df.head()

In [ ]:
def make_trade_time_country(df):
        
    foo = df.reset_index().pivot(index = "time", columns = "CTY_NAME")
    
    foo.drop(labels = ["index"], axis = 1, inplace = True)
    
    return foo

In [ ]:
test = make_trade_time_country(country_df)

In [ ]:
test["total_imports"].drop([ "TOTAL FOR ALL COUNTRIES"], axis = 1)

In [ ]:
def make_time_by_country(df, level):

    height = 533
    width = 625

    foobar = make_trade_time_country(df)
    
    if level == "total_imports":
        title = "By Source Country, $ US Imports "
        
        foobar.drop([ "TOTAL FOR ALL COUNTRIES"], axis = 1, level = 1, inplace = True)
    
    if level == "growth":
        title = "By Source Country, % Year over Year Change in US Imports "
    
    p = figure(plot_height=height, plot_width = width, x_axis_type="datetime",toolbar_location = 'below',
           tools = "box_zoom, reset, pan, xwheel_zoom", title = title, x_range = (dt.datetime(2019,1,1),dt.datetime(2020,6,1)) ) 

    numlines=len(foobar[level].columns)

    multi_line_source = ColumnDataSource({
        'xs': [foobar.index.values]*numlines,
        'ys': [foobar[level, name].values for name in foobar[level]],
        'label': [name for name in foobar["total_imports"].columns]})

    p.multi_line(xs= "xs",
                ys= "ys",
                line_width=2.5, line_alpha=0.45, line_color = "slategray",
                 hover_line_alpha=0.75, hover_line_width = 5,
                hover_line_color= "crimson", source = multi_line_source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)
            
    TIMETOOLTIPS = """
            <div style="background-color:#F5F5F5; opacity: 0.95; border: 5px 5px 5px 5px;">
            <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold"> @label
             </span>
             </div>
             <div style = "text-align:left;">"""
    
    if level == "total_imports":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{$0.0a}</span>   
            </div>
            </div>
            """
    if level == "growth":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{0.0}%</span>   
            </div>
            </div>
            """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS,  line_policy='nearest', formatters={'$data_x': 'datetime'}))
    p.title.text_font_size = '13pt'
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75
    p.border_fill_color = background 
    
    #tradewar_box = BoxAnnotation(left=dt.datetime(2018,7,1), right=dt.datetime(2019,10,11), fill_color='red', fill_alpha=0.1)
    #p.add_layout(tradewar_box)
    
    pandemic_box = BoxAnnotation(left=dt.datetime(2020,3,1), right=dt.datetime(2020,12,31), fill_color='red', fill_alpha=0.1)
    p.add_layout(pandemic_box)
    
    #p.yaxis.axis_label = 
    p.yaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = "13px"

    p.yaxis.minor_tick_line_color = None
    #p.y_range.start = 0
    
    #p.x_range.max_interval = dt.datetime(2020,6,1)
    
    #description = """Each line is a major product catagory in Annex 6-1 of The Agreement."""
    #p.add_layout(Title(text=description, text_font_style="normal", text_font_size="9pt"), 'below')
    #description = "Red marks the period of Section 301 tariffs and retaliation. Blue is period of agreement."""
    #p.add_layout(Title(text=description, text_font_style="normal", text_font_size="9pt"), 'below')

    div0 = Div(text = """Right panel: Each line is an imports country-source. Only the top 10 imports destinations are shown. Left panel:
    Each line is a 3 digit NAICS catagory; only the top 10 catagories are shown. Red is the start of the NBER
    determined recession and Covid-19 pandemic in the US.""", max_width=555, background = background )
    div0.sizing_mode= "scale_both"
    
    if level == "total_imports":
        p.yaxis.formatter = NumeralTickFormatter(format="($0. a)")
    
    p.outline_line_color = None
    #p.WheelZoomTool.maintain_focus = False
    #print(p.y_range.end)
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    #p.WheelZoomTool.maintain_focus = False
    #print(p.y_range.end)
    p = column(p,div0,
                sizing_mode="scale_both")

    return p

In [ ]:
def make_panel_fig():
    
    plevel = make_time_by_product(productdf, "total_imports")
    
    pchange = make_time_by_product(productdf, "growth")
    
    #################################################################33
    
    clevel = make_time_by_country(country_df, "total_imports")
    
    cchange = make_time_by_country(country_df, "growth")
    
    #################################################################
    
    tab1 = Panel(child= plevel, title="Imports in $")

    tab2 = Panel(child= pchange, title="Imports as % YoY")
        
    tab3 = Panel(child= clevel, title="Imports in $")

    tab4 = Panel(child= cchange, title="Imports as % YoY")
        
    outfig = row(Tabs(tabs=[tab4,  tab3], tabs_location = "above"), Tabs(tabs=[tab2,  tab1], tabs_location = "above"))
    
    return outfig

In [ ]:
output_file('.\\docs\\' + "us_imports.html")

outfig = make_panel_fig()

show(outfig)